# Exercise: Fix Data Race

You can use `__syncthreads()` to synchronize threads within a block:

<img src=https://github.com/NVIDIA/accelerated-computing-hub/blob/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Images/sync.png?raw=1 alt="Sync" width=800>

Fix the data race using thread-block synchronization.
Optionally, switch to `cuda::atomic_ref` to reduce the scope of communication:

<details>
<summary>Original code if you need to refer back to it.</summary>

```c++
%%writefile Sources/sync.cpp
#include "ach.cuh"

constexpr float bin_width = 10;

// 1. Use `__syncthreads()` to synchronize threads within a block and avoid data race
__global__ void histogram_kernel(
  cuda::std::span<float> temperatures,
  cuda::std::span<int> block_histograms,
  cuda::std::span<int> histogram)
{
  cuda::std::span<int> block_histogram =
    block_histograms.subspan(blockIdx.x * histogram.size(),
                             histogram.size());

  int cell = blockIdx.x * blockDim.x + threadIdx.x;
  int bin = static_cast<int>(temperatures[cell] / bin_width);

  cuda::std::atomic_ref<int> block_ref(block_histogram[bin]);
  block_ref.fetch_add(1);

  if (threadIdx.x < histogram.size()) {
    // 2. Reduce scope of atomic operation using `cuda::atomic_ref`
    cuda::std::atomic_ref<int> ref(histogram[threadIdx.x]);
    ref.fetch_add(block_histogram[threadIdx.x]);
  }
}


void histogram(
  cuda::std::span<float> temperatures,
  cuda::std::span<int> block_histograms,
  cuda::std::span<int> histogram,
  cudaStream_t stream)
{
  int block_size = 256;
  int grid_size = cuda::ceil_div(temperatures.size(), block_size);
  histogram_kernel<<<grid_size, block_size, 0, stream>>>(
    temperatures, block_histograms, histogram);
}
```

</details>

In [2]:
import os

if os.getenv("COLAB_RELEASE_TAG"): # If running in Google Colab:
  !mkdir -p Sources
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/ach.cuh -nv -O Sources/ach.cuh
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/__init__.py -nv -O Sources/__init__.py
  !wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/ach.py -nv -O Sources/ach.py

2025-12-27 07:10:32 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/ach.cuh [6176/6176] -> "Sources/ach.cuh" [1]
2025-12-27 07:10:32 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/__init__.py [19/19] -> "Sources/__init__.py" [1]
2025-12-27 07:10:32 URL:https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/03.04-Synchronization/Sources/ach.py [3685/3685] -> "Sources/ach.py" [1]


In [7]:
%%writefile Sources/sync.cpp
#include "ach.cuh"

constexpr float bin_width = 10;

__global__ void histogram_kernel(
  cuda::std::span<float> temperatures,
  cuda::std::span<int> block_histograms,
  cuda::std::span<int> histogram)
{
  // TODO: Find where to use `__syncthreads()` to synchronize threads within a
  // block and avoid data race
  cuda::std::span<int> block_histogram =
    block_histograms.subspan(blockIdx.x * histogram.size(),
                             histogram.size());

  int cell = blockIdx.x * blockDim.x + threadIdx.x;
  int bin = static_cast<int>(temperatures[cell] / bin_width);

  cuda::atomic_ref<int,cuda::thread_scope_block> block_ref(block_histogram[bin]);
  block_ref.fetch_add(1);
  __syncthreads();
  if (threadIdx.x < histogram.size()) {
    // TODO: Reduce scope of atomic operation using `cuda::atomic_ref`
    cuda::atomic_ref<int,cuda::thread_scope_device> ref(histogram[threadIdx.x]);
    ref.fetch_add(block_histogram[threadIdx.x]);
  }
}


void histogram(
  cuda::std::span<float> temperatures,
  cuda::std::span<int> block_histograms,
  cuda::std::span<int> histogram,
  cudaStream_t stream)
{
  int block_size = 256;
  int grid_size = cuda::ceil_div(temperatures.size(), block_size);
  histogram_kernel<<<grid_size, block_size, 0, stream>>>(
    temperatures, block_histograms, histogram);
}

Overwriting Sources/sync.cpp


In [ ]:
import Sources.ach
Sources.ach.run("Sources/sync.cpp")

If you’re unsure how to proceed, consider expanding this section for guidance. Use the hint only after giving the problem a genuine attempt.

<details>
  <summary>Hints</summary>
  
  - You can synchronize threads within a thread block using `__syncthreads()`
  - You need to synchronize after all threads have incorporated their changes to the block histogram
  - `cuda::atomic_ref` has exactly the same interface as `cuda::std::atomic_ref` with a difference of accepting thread scope as a second template parameter
</details>

Open this section only after you’ve made a serious attempt at solving the problem. Once you’ve completed your solution, compare it with the reference provided here to evaluate your approach and identify any potential improvements.

<details>
  <summary>Solution</summary>

  Key points:

  - Synchronize before reading the block histogram

  Solution:
  ```c++
  cuda::std::span<int> block_histogram =
    block_histograms.subspan(blockIdx.x * histogram.size(), histogram.size());

  int cell = blockIdx.x * blockDim.x + threadIdx.x;
  int bin = static_cast<int>(temperatures[cell] / bin_width);

  cuda::atomic_ref<int, cuda::thread_scope_block>
    block_ref(block_histogram[bin]);
  block_ref.fetch_add(1);
  __syncthreads();

  if (threadIdx.x < histogram.size())
  {
    cuda::atomic_ref<int, cuda::thread_scope_device>
      ref(histogram[threadIdx.x]);
    ref.fetch_add(block_histogram[threadIdx.x]);
  }
  ```

  You can find full solution [here](Solutions/sync.cpp).
</details>

---
Great job!  Move on to the [next section](../03.05-Shared-Memory/03.05.01-Shared.ipynb).